In [108]:
import sys
from lxml import etree
from time import gmtime, strftime
from datetime import date, timedelta
import re
import pymongo
from pymongo import MongoClient
import time
import os
from ftplib import FTP
import pprint
import requests

In [109]:
fname = './playa.xml'

def process_xml_playa(fname, doc, hay_medusa,info_cruz_roja):
    today = time.strftime("%Y-%m-%d")
    tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")
    
    id_playa_largo = "somo-loredo-3906102"
    nombre_playa = doc["nombre_playa"]
    localidad = doc["localidad"]
    fecha_hoy = today
    fecha_maniana = tomorrow
    estado_cielo_maniana_hoy = doc["cielo_manana"]
    estado_cielo_tarde_hoy = doc["cielo_tarde"]
    estado_cielo_maniana_maniana = doc["m_cielo_manana"]
    estado_cielo_tarde_maniana = doc["m_cielo_tarde"]
    viento_maniana_hoy = doc["viento_manana"]
    viento_tarde_hoy = doc["viento_tarde"]
    oleaje_maniana_hoy = doc["oleaje_manana"]
    oleaje_tarde_hoy = doc["oleaje_tarde"]
    temperatura_maxima_hoy = doc["temperatura_maxima"]
    temperatura_maxima_maniana = doc["m_temperatura_maxima"]
    indice_uv_hoy = doc["indice_uv"]
    temperatura_agua_hoy= doc["temperatura_agua"]
    temperatura_agua_maniana=doc["m_temperatura_agua"]
    pleamar = doc["pleamar"]
    bajamar = doc["bajamar"]
    id_largo = doc["id_largo"]
    info_cruz_roja = info_cruz_roja
    if hay_medusa:
        bandera_hoy = doc["bandera"]
        
    else:
        bandera_hoy = ""
        
   
    hay_medusas = hay_medusa
    enlace_especial = "#"
    enlace_noticia = "#"
    
    f = open(fname)
    
    tree = etree.parse(f)
    root = tree.getroot()
    
    #URL
    txt_url = "http://desn.lasprovincias.es/sociedad/playas/playa.html"
    urls = root.findall('.//url', root.nsmap)
    for url in urls:
        url.text = txt_url
    #Alternate
    urls = root.findall('.//alternate', root.nsmap)
    for url in urls:
        url.text = txt_url
    #Alternate
    
    txt_titulo = "Toda la información sobre el estado de la playa de *NOMBREPLAYA*, en *LOCALIDAD*."
    txt_titulo = txt_titulo.decode('utf8')
    txt_titulo = txt_titulo.replace("*LOCALIDAD*",localidad)
    txt_titulo = txt_titulo.replace("*NOMBREPLAYA*",nombre_playa)
    txt_titulo = txt_titulo
    
    #Titulo (SEO)
    titulos = root.findall('.//Titulo', root.nsmap)
    for titulo in titulos:
        titulo.text = txt_titulo
    
    #Otro Titulo (SEO)
    titulos = root.findall('.//Title', root.nsmap)
    for titulo in titulos:
        titulo.text = txt_titulo
            
    #Titulo noticia
    titulos = root.findall('.//headline', root.nsmap)
    for titulo in titulos:
        titulo.text = txt_titulo
    
    txt_subtitulo = "Datos de temperatura, estado del mar y mareas para que decidas dónde ir"
    txt_subtitulo = txt_subtitulo.decode('utf8')
    
    #Subtítulo SEO
    subtitulos = root.findall('.//subhead', root.nsmap)
    for subtitulo in subtitulos:    
                             

        subtitulo.text = txt_subtitulo
        
    #Subtítulo 
    subtitulos = root.findall('.//subhead', root.nsmap)
    for subtitulo in subtitulos:
        subtitulo.text = txt_subtitulo
    
    #Autor (SEO)print txt_titulo
    txt_autor = "infoplayas/".decode('utf8') + localidad
    txt_autor = txt_autor
    
    autores =  root.findall('.//autor', root.nsmap)
    for autor in autores:
        autor.text = txt_autor
        
    #Más autor#
    autores =  root.findall('.//author/name', root.nsmap)
    for autor in autores:
        autor.text = txt_autor
       
    #Fecha
    fechas =  root.findall('.//fecha', root.nsmap)
    for fecha in fechas:
        fecha.text = strftime("%d/%m/%Y %H:%M", gmtime())
    #Año
    Anios =  root.findall('.//Anio', root.nsmap)
    for Anio in Anios:
        Anio.text = strftime("%Y", gmtime())
    
    url_aemet = ""
    
    archivo = open('./texto_lasprovincias.txt', 'r')
    lasprovincias = archivo.read().decode('utf-8')
    archivo_medusas = open('./bloque_medusas.txt', 'r')
    bloque_medusas = archivo_medusas.read().decode('utf-8')
    archivo_bajamar = open('./bloque_bajamar.txt', 'r')
    bloque_bajamar = archivo_bajamar.read().decode('utf-8')
    
    lasprovincias = lasprovincias.replace('*NOMBRE_PLAYA*', nombre_playa)
    lasprovincias = lasprovincias.replace('*LOCALIDAD*', localidad)
    lasprovincias = lasprovincias.replace('*FECHA_HOY*', fecha_hoy)
    lasprovincias = lasprovincias.replace('*ESTADO_CIELO_HOY_MANIANA*', estado_cielo_maniana_hoy)
    lasprovincias = lasprovincias.replace('*ESTADO_CIELO_HOY_TARDE*', estado_cielo_tarde_hoy)
    lasprovincias = lasprovincias.replace('*URL_AEMET*', id_playa_largo)
    lasprovincias = lasprovincias.replace('*VIENTO_HOY_MANIANA*', viento_maniana_hoy)
    lasprovincias = lasprovincias.replace('*VIENTO_HOY_TARDE*', viento_tarde_hoy)
    lasprovincias = lasprovincias.replace('*OLEAJE_HOY_MANIANA*', oleaje_maniana_hoy)
    lasprovincias = lasprovincias.replace('*OLEAJE_HOY_TARDE*', oleaje_tarde_hoy)
    lasprovincias = lasprovincias.replace('*TEMPERATURA_MAXIMA_HOY*', str(temperatura_maxima_hoy))
    lasprovincias = lasprovincias.replace('*INDICE_UV_HOY*', str(indice_uv_hoy))
    if indice_uv_hoy <= 2:
        indice = "baja"
    elif indice_uv_hoy >= 3 and indice_uv_hoy <= 5:
        indice = "media"
    elif indice_uv_hoy >= 6 and indice_uv_hoy <= 7:
        indice = "alta"
    elif indice_uv_hoy >= 8 and indice_uv_hoy <= 10:
        indice = "muy alta"
    else:
        indice = "extremadamente alta"
    lasprovincias = lasprovincias.replace('*EXPOSICION_HOY*', indice)
    lasprovincias = lasprovincias.replace('*TEMPERATURA_AGUA_HOY*', str(temperatura_agua_hoy))
    if bajamar !='':
        bloque_bajamar = bloque_bajamar.replace('*BAJAMAR*', bajamar)
        bloque_bajamar = bloque_bajamar.replace('*pleamar*', pleamar)
    else:
        bloque_bajamar = ""
    lasprovincias = lasprovincias.replace('*BLOQUE_BAJAMAR*', bloque_bajamar)
    
    if info_cruz_roja:
        condiciones = ""
        bandera = bandera_hoy
        texto_medusa = ""
        bloque_medusas = bloque_medusas.replace('*BANDERA*', bandera)
        bloque_medusas = bloque_medusas.replace('*CONDICIONES*', condiciones)
        bloque_medusas = bloque_medusas.replace('*TEXTO_MEDUSA*', texto_medusa)
    else:
        bloque_medusas= ""
    #[No se ha detectado presencia de medusas / Atención, se ha detectado la presencia de medusas].
    lasprovincias = lasprovincias.replace('*BLOQUE_MEDUDUSAS*', bloque_medusas)
        
    lasprovincias = lasprovincias.replace('*FECHA_MANIANA*', fecha_maniana)
    lasprovincias = lasprovincias.replace('*CIELO_MANIANA_MANIANA*', estado_cielo_maniana_maniana)
    lasprovincias = lasprovincias.replace('*CIELO_MANIANA_TARDE*', estado_cielo_tarde_maniana)
    lasprovincias = lasprovincias.replace('*TEMPERATURA_MAXIMA_MANIANA*', str(temperatura_maxima_maniana))
    lasprovincias = lasprovincias.replace('*ID_LARGO*', id_largo)
    
    text = "<![CDATA[" + lasprovincias + "]]>"
    
    
    #Año
    textos =  root.findall('.//story/text', root.nsmap)
    for t in textos:
        t.text = text
    
    f.close()
    return etree.tostring(root, pretty_print=True)

#def main():
#    fname = './playa.xml'
#    text = process_xml_playa(fname)
#    outfile = open('./out.xml', 'w+')
#    outfile.write(text)
#    outfile.close()


#if __name__ == '__main__':
#    main()

def normalize_text(text):
    '''
        Elimina signos de puntuación
        Elimina tildes
        Cambia espacios por guiones
        Convierte la cadena a minúscula
    '''
    import unicodedata

    norm_text = ''.join((c for c in unicodedata.normalize('NFD',text.decode('utf-8')) if unicodedata.category(c) != 'Mn'))
    norm_text_out = ''
    hyphen = 0
    for char in norm_text.encode('latin1'):
        if str.isalnum(char):
            norm_text_out += char
            hyphen = 0
        else:
            if hyphen == 0:
                norm_text_out += '-'
                hyphen = 1
    return norm_text_out.strip('-').lower()


In [124]:
client = MongoClient()
client = MongoClient('localhost', 27017)

db_aemet = client.AEMET
db_croja = client.CRUZROJA
    
    
col_playas = db_croja['playas']
col_localidades = db_aemet['localidades']
col_costas = db_aemet['costas']
today = time.strftime("%Y-%m-%d")
tomorrow = (date.today() + timedelta(days=1)).strftime("%Y-%m-%d")
today_costa = today + " 00:00:00"
tomorrow_costa = tomorrow + " 00:00:00"
info_cruz_roja = True

#Para probar si carga uno o el otro
#comunidad = "cantabria"
comunidad = "valencia"
#Hago una query de comprobación (saco unicamente un registro):
#Cantabria

if comunidad =="cantabria":
    playas  = col_playas.find({"id_costa": { "$exists" : True }, "provincia": {"$in": ["Cantabria"]}},
                             {"nombre_playa":1,
                              "id_costa":1,
                              "municipio":1,
                              "provincia":1,
                              "latitud":1,
                              "longitud":1,
                               today + ".medusas": 1,
                               today + ".bandera":1})

    
#Valencia
else:
    playas  = col_playas.find({"id_costa": { "$exists" : True }, "provincia": {"$in": ["Castellon", "Alicante", "Valencia"]}},
                                 {"nombre_playa":1,
                                  "id_costa":1,
                                  "municipio":1,
                                  "provincia":1,
                                  "latitud":1,
                                  "longitud":1,
                                   today + ".medusas": 1,
                                   today + ".bandera":1})

paths = "/path/to/xml"

#Para las playas
for playa in playas:
    doc = {}
    id_costa = str(playa["id_costa"]).zfill(7)
    costas = col_costas.find({"_id": unicode(id_costa)},
                             {today_costa : 1,
                              tomorrow_costa:1,
                              "nombre_provincia":1,
                              "id_largo":1
                             })

    #Saco la info del raiz de playas
    doc['idplaya'] = playa['_id']
    doc['nombre_playa'] = playa["nombre_playa"]
    id_largo = str(playa['_id'])+"-"+normalize_text(playa['provincia'].encode('utf8')) +"-"+ normalize_text(playa['nombre_playa'].encode('utf8'))
                                                     
    doc["localidad"] = playa["municipio"]
    doc["provincia"] = playa["provincia"]
    doc['latitud'] = playa['latitud']
    doc['longitud'] =  playa['longitud']
    #print "############################"
    #pprint.pprint(playa)
    for costa in costas:
        doc['viento_tarde'] = costa[today_costa]['viento_tarde']
        if 'pleamar' in costa[today_costa]:
            doc['pleamar'] = costa[today_costa]['pleamar']
            doc['bajamar'] =  costa[today_costa]['bajamar']
        else:
            doc['pleamar'] = ""
            doc['bajamar'] =  ""
        doc['temperatura_agua'] =  costa[today_costa]['temperatua_agua']
        doc['cielo_tarde'] =  costa[today_costa]['cielo_tarde']
        doc['temperatura_maxima'] =  costa[today_costa]['temperatura_maxima']
        doc['viento_manana'] =  costa[today_costa]['viento_manana']
        doc['oleaje_manana'] =  costa[today_costa]['oleaje_manana']
        doc['indice_uv'] =  costa[today_costa]['indice_uv']
        doc['oleaje_tarde'] =  costa[today_costa]['oleaje_tarde']
        doc['sensacion_termica'] =  costa[today_costa]['sensacion_termica']
        doc['cielo_manana'] =  costa[today_costa]['cielo_manana']
        doc['m_viento_tarde'] = costa[tomorrow_costa]['viento_tarde']
        doc['m_temperatura_agua'] =  costa[tomorrow_costa]['temperatua_agua']
        doc['m_cielo_tarde'] =  costa[tomorrow_costa]['cielo_tarde']
        doc['m_temperatura_maxima'] =  costa[tomorrow_costa]['temperatura_maxima']
        doc['m_viento_manana'] =  costa[tomorrow_costa]['viento_manana']
        doc['m_oleaje_manana'] =  costa[tomorrow_costa]['oleaje_manana']
        doc['m_indice_uv'] =  costa[tomorrow_costa]['indice_uv']
        doc['m_oleaje_tarde'] =  costa[tomorrow_costa]['oleaje_tarde']
        doc['m_sensacion_termica'] =  costa[tomorrow_costa]['sensacion_termica']
        doc['m_cielo_manana'] =  costa[tomorrow_costa]['cielo_manana']
        doc['id_largo'] = id_largo
    
 
    hay_medusa = False
    
    #Compruebo si tenemos infor de medusas
    if today in playa:
        doc['medusas'] = playa[today]['medusas']
        doc['badera'] = playa[today]['bandera']
        hay_medusa = True
        info_cruz_roja = False
    
    doc['path'] = "/sociedad/playas/%s.html" % id_largo
    


    text = process_xml_playa(fname,doc,hay_medusa,info_cruz_roja)
    
    outfile = open('./xml/' + comunidad + "/" + str(id_largo)+ '.xml', 'w+')
    outfile.write(text)
    outfile.close()

    #hago el xml
    
    
######################################################################
# Para las costas
#
# Costas no CRUZROJA
#
# Número de playas de la cruz roja
######################################################################
#Cantabria
if comunidad == 'cantabria':
    resultado_playas = col_playas.find({"id_costa": {"$exists": True}, "provincia": {"$in": ["Cantabria"]}})
else:
    #Valencia
    resultado_playas = col_playas.find({"id_costa": {"$exists": True}, "provincia": {"$in": ["Castellon", "Alicante", "Valencia"]}})
ids_costa = []
for playa in resultado_playas:
    ids_costa.append(playa['id_costa'])

#número de playas de la aemet
if comunidad == 'cantabria':
    resultado_costas = col_costas.find({"_id": {"$nin":ids_costa}, "id_provincia": {"$in": ["39"]}},
                                  {today_costa : 1,
                                   tomorrow_costa:1,
                                   "nombre_provincia":1,
                                   "id_largo":1,
                                   "_id":1,
                                   "nombre_playa":1,
                                   "nombre_localidad":1,
                                   "nombre_provincia":1,
                                   "latitud":1,
                                   "longitud":1
                                  })
else:
    resultado_costas = col_costas.find({"_id": {"$nin":ids_costa}, "id_provincia": {"$in": ["03", "12", "46"]}},
                                  {today_costa : 1,
                                   tomorrow_costa:1,
                                   "nombre_provincia":1,
                                   "id_largo":1,
                                   "_id":1,
                                   "nombre_playa":1,
                                   "nombre_localidad":1,
                                   "nombre_provincia":1,
                                   "latitud":1,
                                   "longitud":1
                                  })
for costa in resultado_costas:
    doc = {}
    #Saco la info del raiz de playas
    doc['idplaya'] = costa['_id']
    doc['nombre_playa'] = costa["nombre_playa"]
    doc["localidad"] = costa["nombre_localidad"]
    doc["provincia"] = costa["nombre_provincia"]
    doc['latitud'] = costa['latitud']
    doc['longitud'] =  costa['longitud']
    doc['viento_tarde'] = costa[today_costa]['viento_tarde']
    if 'pleamar' in costa[today_costa]:
        doc['pleamar'] = costa[today_costa]['pleamar']
        doc['bajamar'] =  costa[today_costa]['bajamar']
    else:
        doc['pleamar'] = ""
        doc['bajamar'] =  ""
    doc['temperatura_agua'] =  costa[today_costa]['temperatua_agua']
    doc['cielo_tarde'] =  costa[today_costa]['cielo_tarde']
    doc['temperatura_maxima'] =  costa[today_costa]['temperatura_maxima']
    doc['viento_manana'] =  costa[today_costa]['viento_manana']
    doc['oleaje_manana'] =  costa[today_costa]['oleaje_manana']
    doc['indice_uv'] =  costa[today_costa]['indice_uv']
    doc['oleaje_tarde'] =  costa[today_costa]['oleaje_tarde']
    doc['sensacion_termica'] =  costa[today_costa]['sensacion_termica']
    doc['cielo_manana'] =  costa[today_costa]['cielo_manana']
    doc['m_viento_tarde'] = costa[tomorrow_costa]['viento_tarde']
    doc['m_temperatura_agua'] =  costa[tomorrow_costa]['temperatua_agua']
    doc['m_cielo_tarde'] =  costa[tomorrow_costa]['cielo_tarde']
    doc['m_temperatura_maxima'] =  costa[tomorrow_costa]['temperatura_maxima']
    doc['m_viento_manana'] =  costa[tomorrow_costa]['viento_manana']
    doc['m_oleaje_manana'] =  costa[tomorrow_costa]['oleaje_manana']
    doc['m_indice_uv'] =  costa[tomorrow_costa]['indice_uv']
    doc['m_oleaje_tarde'] =  costa[tomorrow_costa]['oleaje_tarde']
    doc['m_sensacion_termica'] =  costa[tomorrow_costa]['sensacion_termica']
    doc['m_cielo_manana'] =  costa[tomorrow_costa]['cielo_manana']
    doc['id_largo'] = costa["id_largo"]
    id_largo = doc['id_largo']
    hay_medusa = False
    info_cruz_roja = False

    doc['path'] = "/sociedad/playas/%s.html" % id_largo
    


    text = process_xml_playa(fname,doc,hay_medusa,info_cruz_roja)
    
    outfile = open('./xml/' + comunidad + "/" + str(id_largo)+ '.xml', 'w+')
    outfile.write(text)
    outfile.close()
